In [1]:
import os
import ipyparallel
import sys
import numpy as np
sys.path.insert(0,"/Users/rgerkin/dev/neuroml/pyNeuroML")
sys.path.insert(0,"/Users/rgerkin/dev/scidash/neuronunit")

In [2]:
c = ipyparallel.Client()
dview = c[:] # The main object

In [3]:
with dview.sync_imports(): # Causes each of these things to be imported on the workers as well as here.  
    import matplotlib
    
def f():
    matplotlib.use('Agg') # Need to do this before importing neuronunit on a Mac, because OSX backend won't work
                          # on the worker threads.
dview.apply_sync(f);

importing matplotlib on engine(s)


In [4]:
with dview.sync_imports(): # Causes each of these things to be imported on the workers as well as here.  
    import sciunit
    import neuronunit
import sciunit.scores
import numpy as np

importing sciunit on engine(s)
importing neuronunit on engine(s)


In [5]:
# A trivial model just to demonstrate ipyparallel.
class MyModel(sciunit.Model):
    def get_membrane_potential(self):
        return self.params['vr']
    def inject_square_current(self,injected):
        pass
    
# Make a bunch of models like this with different resting potentials.  
models = []
for vr in np.linspace(-80,-50,4):
    model = MyModel(name='V_rest=%dmV' % vr,vr=vr)
    models.append(model)

In [6]:
# A trivial test just to demonstrate ipyparallel.
class MyTest(sciunit.Test):
    score_type = sciunit.scores.FloatScore
    def generate_prediction(self,model):
        x = model.get_membrane_potential()
        return x
    def compute_score(self,prediction,observation):
        return self.score_type(prediction-observation)
test = MyTest(observation=-65)

In [7]:
# Serial execution of this trivial model without ipyparallel, for reference.  
test.judge(models)

,MyTest
V_rest=-80mV,15
V_rest=-70mV,5
V_rest=-60mV,-5
V_rest=-50mV,-15


In [8]:
# Same as above, but with ipyparallel.  
dview.push({'MyTest':MyTest,'MyModel':MyModel}) # These classes need to be copied to the workers' namespaces.  
scores = dview.map_sync(test.judge, models) # The map function, returning scores.  
sciunit.ScoreMatrix([test],models,scores=scores) # Assemble the scores back into a score matrix.  

,MyTest
V_rest=-80mV,15
V_rest=-70mV,5
V_rest=-60mV,-5
V_rest=-50mV,-15


In [9]:
# A real test.  
#with dview.sync_imports(): # Not sure how many of these actually need to be imported by the workers.  
from neuronunit import tests as nu_tests, neuroelectro

neuron = {'nlex_id': 'nifext_50'} # Layer V pyramidal cell
cls = nu_tests.RestingPotentialTest
observation = cls.neuroelectro_summary_observation(neuron)
test = cls(observation)

Getting cached Resting membrane potential data values from neuroelectro.org
Using cached value.


In [11]:
# A real model.  
#with dview.sync_imports(): # Not sure how many of these actually need to be imported by the workers.  
from neuronunit.models.reduced import ReducedModel
#from neuronunit.models.backends import jNeuroMLBackend

DOCS_PATH = os.getcwd()+'/docs' # Assume this notebook is being executed from inside the neuronunit/docs directory.
assert DOCS_PATH.endswith('neuronunit/docs'), "Change the DOCS_PATH variable to the path to your neuronunit/docs directory"
LEMS_MODEL_PATH = os.path.join(DOCS_PATH,'data/NeuroML2/LEMS_2007One.xml')

# We make this version of the model (setting jNeuroMLBackend by multiple inheritance)
# because otherwise the version of the ReducedModel class that gets pushed to the workers
# will be the vanilla one as defined in neuronunit rather than the one patched with the backend at runtime.  
# The vanilla one does not have certain capabilties implemented (e.g. inject_square_current), which will
# cause an NotImplementedError on the workers.  By adding jNeuroMLBackend here we create a class where this
# method is automatically implemented.  We might consider just implementing backends in this way rather than
# at runtime via set_backend, to avoid these kinds of problems (which might plague scoop as well).  

models = []
for vr in np.linspace(-80,-50,4):
    model = ReducedModel(LEMS_MODEL_PATH, 
                         name='V_rest=%dmV' % vr, 
                         attrs={'//izhikevich2007Cell':
                                    {'vr':'%d mV' % vr}
                               },)
    models.append(model)

In [12]:
# Now we run the test.  
# I use the synchronous version in which we check to see if the result is finished, 
# because I think trying to get the result too early may actually screw something up.  
scores = dview.map_sync(test.judge, models)

In [13]:
sciunit.ScoreMatrix([test],models,scores=result)

,RestingPotentialTest
V_rest=-80mV,Z = -1.80
V_rest=-70mV,Z = -0.27
V_rest=-60mV,Z = 1.26
V_rest=-50mV,Z = 2.79


In [14]:
# Serial version without ipyparallel
test.judge(models)

/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp5j_3es2h
/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp3ok025g0


/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp9tkvclcc
/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp7_p8ygzz


/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmpxcwfqsat
/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmpsoipixii


/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp4a7vz_0i
/var/folders/fz/8tx2ql0d7273z1kqpkcqcy7w0000gp/T/tmp4kjf6_4t


,RestingPotentialTest
V_rest=-80mV,Z = -1.80
V_rest=-70mV,Z = -0.27
V_rest=-60mV,Z = 1.26
V_rest=-50mV,Z = 2.79
